# ライブラリ

In [ ]:
import numpy as np
import random
from random import Random
import itertools

import unittest
from unittest.mock import patch  # For mocking and patching

#from ipycanvas import Canvas

# Class of nanoword

## Define Classes and functions

In [ ]:
SIGNS = ['a+', 'a-', 'b+', 'b-']
FACTORS = [{'a+', 'b-'}, {'b+', 'a-'}]

def get_factor(sign):
    if not sign in SIGNS: raise ValueError(f"{sign} is not in {SIGNS}")
    return FACTORS[0] if sign in FACTORS[0] else FACTORS[1]

def tau(sign):
    return (get_factor(sign) - {sign}).pop()

def iota(self, sign):
    result = sign.replace('+', '-')
    if result == sign:
        result = sign.replace('-', '+')
    return result

#---------------------------------
ALL_CHARS = [chr(i) for i in  range(ord('A'), ord('Z')+1)]

class Letter:
    def __init__(self, char, sign):
        if type(char) is not str or not len(char) == 1: raise ValueError(f"{char} is not a letter")
        if not sign in SIGNS: raise ValueError(f"{sign} is not in {SIGNS}")
        self.char = char
        self.sign = sign        

    def __eq__(self, other):
        return True if (self.char == other.char and self.sign == self.sign) else False

class Nanoword:
    def __init__(self, word, alphabet) -> None:
        '''
        word <-- a word on alphabet
        alphabet <-- a list of letters
        '''
        self.word = word
        self.size = len(word)
        self.alphabet = alphabet
        self.chars = [l.char for l in self.alphabet]
        self.validation_check()

    def validation_check(self) -> None:
        if not self.is_gauss_word(): raise ValueError(f"{self.word} is not a Gauss word.")
        if not set(self.chars) == set(self.word): raise ValueError(f"The charactors in the alphabet: {self.chars} does not match with the word {self.word}")
        
    def is_gauss_word(self) -> bool:
        '''
        Check the word is a Gauss word or not.
        '''
        return all([(self.word.count(char)==2) for char in self.word])
    
    #---
    def __str__(self) -> str:
        return self.word
    
    def __eq__(self, other) -> bool:
        result = False
        if len(self.alphabet) == len(other.alphabet):
            result = all([(l in other.alphabet) for l in self.alphabet])
        return self.word == other.word and result
        
    def add_letter(self, a_letter):
        pass
        
    #---------#
    def rmi(self, char=None) -> bool:
        result = None
        if char:
            new_word = self.word.replace(char+char,'')
            if len(new_word) < self.size:
                new_alphabet = [l for l in self.alphabet if not l.char == char]
                result = type(self)(new_word, new_alphabet)
        else:
            for c in self.chars:
                result = self.rmi(char=c)
                if result:
                    break
        return result
        
    def rmi_inv(self, letter=None, index=None):
        result = None
        if letter is None:
            remaining_chars = [c for c in ALL_CHARS if c not in self.chars]
            try:
                letter = Letter(remaining_chars[0], 'a+')
            except Exception as e:
                print(e)
        if index is None: 
            index = random.randint(0, self.size)
        #---
        if letter.char not in self.chars:
            c = letter.char
            new_word = self.word[:index]+c+c+self.word[index:]
            new_alphabet = self.alphabet + [letter]
            result = type(self)(new_word, new_alphabet)
        else:
            raise ValueError(f"{letter.char} must not be in the alphabet of this nanoword")
        return result        
        
    def rmii(self, chars=None):
        result = None
        if chars:
            if len(chars) != 2: raise ValueError(f"{chars} are not a pair of characters")
            letters = [l for l in self.alphabet if l.char in chars]
            if len(letters) != 2: raise ValueError(f"{chars} are not in the alphabet of this nanoword")
            if tau(letters[0].sign) == letters[1].sign:
                ll = chars[0]+chars[1]
                new_word = self.word.replace(ll, '').replace(ll[::-1], '')
                new_alphabet = [v for v in self.alphabet if not v in letters]
                result = type(self)(new_word, new_alphabet)
        else:
            for pair in zip(self.word, self.word[1:]):
                result = self.rmii(chars=list(pair))
                if result:
                    break
        return result

    def rmii_inv(self, letters=None, indices=(0,1)):
        result = None
        if letters is None:
            remaining_chars = [c for c in ALL_CHARS if c not in self.chars]
            try:
                letters = [Letter(remaining_chars[0], 'a+'), Letter(remaining_chars[1], 'b-')]
            except Exception as e:
                print(e)
        if indices is None: indices = [rand.randint(0, self.size) for _ in range(2)]
        #---
        if len(letters) == 2 and (not {l.char for l in letters}.issubset(self.chars)) and tau(letters[0].sign) == letters[1].sign:
            fl, sl = letters[0], letters[1]
            pair = fl.char+sl.char
            new_word = self.word[:indices[0]] + pair + self.word[indices[0]:indices[1]] + pair[::-1] + self.word[indices[1]:]
            new_alphabet = self.alphabet + letters
            result = type(self)(new_word, new_alphabet)
        else:
            raise ValueError(f"{letters} are invalid. Must be a pair of letters that are not in the alphabet of this nanoword.")
        return result

    def rmiii(self, letters=None, index=0):
        result = None
        if letters:
            pass
        else:
            pass
        return result

## Unit tests

In [ ]:
random = Random()
class TestReidemeisterMoves(unittest.TestCase):
    def setUp(self):
        self.alph = [Letter("A",'b+'), Letter("B", 'b-'), Letter("C", 'a+'), Letter("D", 'a-'), Letter("E", 'b+'), Letter("F", 'a-')]
        self.nw = Nanoword("ABCDAECBFDFE", self.alph)
        self.ls = [Letter("X", 'a+'), Letter("Y", 'b-'), Letter("Z", 'b+')]
        #---
        global random
        random = Random(666)        
    def tearDown(self):
        del self.alph
        del self.nw
        del self.ls
        
#----------------------
    def test_initialize__not_gauss(self):
        with self.assertRaises(ValueError):
            Nanoword('ABC', self.alph[:3])

    def test_initialize__invalid_alphabet(self):
        with self.assertRaises(ValueError):
            Nanoword('ABCBAC', self.alph[:4])

#----------------------
    def test_equal(self):
        alph = self.alph[:3]
        w1, w2 = Nanoword('ABCCBA', alph), Nanoword('ABCCBA', alph)
        self.assertEqual(w1, w2)
        
    def test_equal_w_dif_words(self):
        alph = self.alph[:3]
        w1, w2 = Nanoword('ABCCBA', alph), Nanoword('AACBCB', alph)
        self.assertNotEqual(w1, w2)
        
#--- Reidemeister I ---
#    @patch("random.randint")
    def test_rmi_inv(self):
#        mock_randint.return_value = 1
        expected = "AGGBCDAECBFDFE" 
        actual = self.nw.rmi_inv().word
        self.assertEqual(expected, actual)
        
    def test_rmi_inv__w_char_and_index(self):
        expected = "ABCDXXAECBFDFE" 
        actual = self.nw.rmi_inv(letter=self.ls[0], index=4).word
        self.assertEqual(expected, actual)
        
    def test_rmi__w_letter(self):
        myw = self.nw.rmi_inv()
        expected = self.nw
        actual = myw.rmi(char='G')
        self.assertEqual(expected, actual)

    def test_rmi__w_letter_o_None_01(self):
        myw = self.nw.rmi_inv()
        self.assertIsNone(myw.rmi(char='F'))

    def test_rmi__w_letter_o_None_02(self):
        self.assertIsNone(self.nw.rmi(char="A"))

#--- Reidemeister II ---
#    @patch("random.randint")
    def test_rmii_inv(self):
#        mock_randint.return_value = [1, 5]
        expected = "AGHBCDAHGECBFDFE" 
        actual = self.nw.rmii_inv().word
        self.assertEqual(expected, actual)
        
    def test_rmii_inv__w_letters_and_indices(self):
        expected = "XYABCDYXAECBFDFE" 
        actual = self.nw.rmii_inv(letters=self.ls[:2], indices=[0,4]).word
        self.assertEqual(expected, actual)
        
    def test_rmii(self):
        expected = self.nw.word.replace('B', '').replace('C', '')
        actual = self.nw.rmii().word
        self.assertEqual(expected, actual)
    # def test_rmii(self):
    #     mynw = Nanoword("XYABCDYXAECBFDFE", self.alph + self.ls[:2])
    #     expected = self.nw
    #     actual = mynw.rmii()
    #     self.assertEqual(expected, actual)

    def test_rmii__w_chars(self):
        expected = self.nw.word.replace('B','').replace('C','')
        actual = self.nw.rmii(chars=['B','C']).word
        self.assertEqual(expected,actual)

    def test_rmii__w_chars_o_None(self):
        actual = self.nw.rmii(chars=['A','B'])
        self.assertIsNone(actual)

#     def test_rmi__w_letter_and_index(self):
#         expected = None
#         actual = self.nw.rmi(char="A", index=2)
#         self.assertEqual(expected, actual)

#--- Reidemeister III ---

# Running the tests

In [ ]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)